In [10]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import evaluate

## Select Device (CPU, CUDA, or MPS if on Apple Silicon)

In [11]:

if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")


Using device: mps


## Load the SST2 Dataset

In [12]:
# The SST2 dataset is part of the GLUE benchmark. 
# We use the "validation" split for demonstration.
dataset_sst2 = load_dataset("glue", "sst2")
dataset = dataset_sst2["validation"]

## Load Model and Tokenizer

In [13]:
model_name = "HuggingFaceTB/SmolLM-1.7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to(device)
model.eval()

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM-1.7B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,)

## Preprocessing Function

In [14]:
def preprocess(examples):
    """
    Tokenize the text from the SST2 dataset.
    SST2 has a "sentence" key. The label is in "label".
    """
    return tokenizer(
        examples["sentence"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

## Preprocess Dataset

In [15]:
encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

## Perform Inference to Obtain Model Outputs

In [16]:
import os

logits_file = 'all_logits.npy'
labels_file = 'all_labels.npy'

if os.path.exists(logits_file) and os.path.exists(labels_file):
    all_logits = np.load(logits_file)
    all_labels = np.load(labels_file)
else:
    all_logits = []
    all_labels = []
    
    with torch.no_grad():
        for i in range(len(encoded_dataset)):
            batch = {
                "input_ids": encoded_dataset[i]["input_ids"].unsqueeze(0).to(device),
                "attention_mask": encoded_dataset[i]["attention_mask"].unsqueeze(0).to(device),
            }
            outputs = model(**batch)
            logits = outputs.logits
            all_logits.append(logits.cpu().numpy()[0])  # move logits back to CPU for processing
            all_labels.append(encoded_dataset[i]["label"].item())
    
    all_logits = np.array(all_logits)
    all_labels = np.array(all_labels)
    np.save(logits_file, all_logits)
    np.save(labels_file, all_labels)

## Convert Logits to Probabilities and Predictions

In [17]:
probabilities = torch.nn.functional.softmax(torch.tensor(all_logits), dim=-1).numpy()
pred_labels = np.argmax(probabilities, axis=-1)

## Compute Accuracy

In [18]:
accuracy_score = np.mean(pred_labels == all_labels)

## Compute Negative Log-Likelihood (NLL)

In [19]:
def compute_nll(probs, labels):
    """
    NLL is defined as -log(p(correct_label)).
    We'll compute the average NLL over the dataset.
    """
    p_correct = probs[np.arange(len(labels)), labels]
    # clip to avoid numerical instability
    p_correct = np.clip(p_correct, 1e-12, 1.0)
    return -np.mean(np.log(p_correct))

nll_score = compute_nll(probabilities, all_labels)

## Compute Expected Calibration Error (ECE)

In [20]:
# A simple ECE implementation using equally spaced bins.
def compute_ece(probs, labels, num_bins=10):
    """
    :param probs: Array of shape (N, num_classes) with predicted probabilities
    :param labels: 1D array of shape (N,) with ground-truth labels
    :param num_bins: Number of confidence bins
    :return: ECE value (float)
    """
    confidences = np.max(probs, axis=-1)
    predictions = np.argmax(probs, axis=-1)
    
    bin_boundaries = np.linspace(0.0, 1.0, num_bins + 1)
    ece = 0.0
    
    for i in range(num_bins):
        start = bin_boundaries[i]
        end = bin_boundaries[i+1]
        mask = (confidences >= start) & (confidences < end)
        if np.sum(mask) == 0:
            continue
        
        bin_accuracy = np.mean(predictions[mask] == labels[mask])
        bin_confidence = np.mean(confidences[mask])
        bin_proportion = np.mean(mask)
        
        ece += abs(bin_confidence - bin_accuracy) * bin_proportion
    
    return ece

ece_score = compute_ece(probabilities, all_labels, num_bins=10)

print(f"Number of examples: {len(dataset)}")
print(f"Accuracy: {accuracy_score:.4f}")
print(f"NLL: {nll_score:.4f}")
print(f"ECE: {ece_score:.4f}")


Number of examples: 872
Accuracy: 0.4908
NLL: 0.7784
ECE: 0.1872


In [25]:

def compute_precision(pred, true):
    tp = np.sum((pred == 1) & (true == 1))
    fp = np.sum((pred == 1) & (true == 0))
    return tp / (tp + fp) if (tp + fp) > 0 else 0.0

def compute_recall(pred, true):
    tp = np.sum((pred == 1) & (true == 1))
    fn = np.sum((pred == 0) & (true == 1))
    return tp / (tp + fn) if (tp + fn) > 0 else 0.0

def compute_f1(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

precision_score = compute_precision(pred_labels, all_labels)
recall_score = compute_recall(pred_labels, all_labels)
f1_score = compute_f1(precision_score, recall_score)

print(f"Precision: {precision_score}")
print(f"Recall: {recall_score}")
print(f"F1-score: {f1_score}")

Precision: 0.0
Recall: 0.0
F1-score: 0.0


In [26]:
def precision_recall_f1(y_true, y_pred, positive_label=1):
    """
    Calculate precision, recall, and F1 score for binary classification.

    Parameters:
    - y_true: list or array of true labels.
    - y_pred: list or array of predicted labels.
    - positive_label: the value in the labels that is considered the positive class (default is 1).

    Returns:
    - precision: float, precision score.
    - recall: float, recall score.
    - f1_score: float, F1 score.
    """
    # Initialize counts for True Positives, False Positives, and False Negatives.
    TP = FP = FN = 0

    # Loop over true and predicted labels simultaneously to compute TP, FP, and FN.
    for true, pred in zip(y_true, y_pred):
        if pred == positive_label:
            if true == positive_label:
                TP += 1  # Correctly predicted positive.
            else:
                FP += 1  # Incorrectly predicted positive.
        else:
            if true == positive_label:
                FN += 1  # Positive instance missed by prediction.

    # Calculate precision, ensuring we do not divide by zero.
    if TP + FP > 0:
        precision = TP / (TP + FP)
    else:
        precision = 0.0

    # Calculate recall, ensuring we do not divide by zero.
    if TP + FN > 0:
        recall = TP / (TP + FN)
    else:
        recall = 0.0

    # Calculate F1 score if both precision and recall are non-zero.
    if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0.0

    return precision, recall, f1_score




precision, recall, f1 = precision_recall_f1(all_labels, pred_labels)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0.0
Recall: 0.0
F1 Score: 0.0


In [27]:
print(all_labels),print(pred_labels)

[1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1 1 1 0 0 0 0 0 1 1 0 0 1 0 0 1 0 1 0 0 0 1
 0 1 1 1 1 1 1 0 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 0 1 1 0 0 1 1 1 0 0 1 1 1
 0 1 0 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 1 0 1 0 0 1 0
 0 1 0 1 0 1 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0
 0 1 1 1 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0
 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0 1 1 1 1 0 0 1 0 0 0 0 1 1 0 0 1 1 0
 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0 0 0
 1 0 1 0 1 0 0 1 0 0 0 1 1 0 1 0 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 1 0 0 1 1 1
 0 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1 0 0 1 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 1 1 1 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1
 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 0 0 1 0 1
 0 0 0 1 0 0 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1 1 0 0 1
 0 1 0 1 1 0 1 1 0 1 0 1 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 1 1 0 1 0 0 0 1 1
 1 1 1 1 1 0 0 1 0 0 0 1 

(None, None)